---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [11]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder

#def plot_graph(G, weight_name=None):
#'''
#    G: a networkx G
#    weight_name: name of the attribute for plotting edge weights (if G is weighted)
#'''
#    %matplotlib notebook
#    import matplotlib.pyplot as plt
    
#    plt.figure()
#    pos = nx.spring_layout(G)
#    edges = G.edges()
#    weights = None
    
#    if weight_name:
#        weights = [int(G[u][v][weight_name]) for u,v in edges]
#        labels = nx.get_edge_attributes(G,weight_name)
#        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
#        nx.draw_networkx(G, pos, edges=edges, width=weights);
#    else:
#        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [12]:
def answer_one():
     
    df = pd.read_csv('Employee_Movie_Choices.txt', sep='\t')
    G = nx.from_pandas_dataframe(df, '#Employee', 'Movie')
    
    return G

In [13]:
answer_one().nodes()

['Andy',
 'Anaconda',
 'Mean Girls',
 'The Matrix',
 'Claude',
 'Monty Python and the Holy Grail',
 'Snakes on a Plane',
 'Frida',
 'The Shawshank Redemption',
 'The Social Network',
 'Georgia',
 'Joan',
 'Forrest Gump',
 'Kung Fu Panda',
 'Lee',
 'Pablo',
 'The Dark Knight',
 'Vincent',
 'The Godfather']

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [14]:
def answer_two():
    
    G = answer_one()
    for node in G.nodes():
        if node in employees:
            G.add_node(node, type='employee')
        elif node in movies:
            G.add_node(node, type='movie')
    return G

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [15]:
def answer_three():
        
    G = answer_two()
    G_weighted = bipartite.weighted_projected_graph(G, employees)
    
    return G_weighted

In [65]:
answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [63]:
def answer_four():
        
    G = answer_three() # load weighted bipartite
    rel = nx.read_edgelist('Employee_Relationships.txt', data=[('relationship_score', int)]) # load relationship
    
    G_df = pd.DataFrame(G.edges(data=True), columns = ['From','To','movie_score'])
    G_df['movie_score'] = G_df['movie_score'].map(lambda x: x['weight'])
    G_df_2 = G_df.copy()
    ls_G_df = list(G_df_2)
    ls_G_df[0], ls_G_df[1] =ls_G_df[1],ls_G_df[0]
    G_df_2.columns = ls_G_df
    G_df = pd.concat([G_df, G_df_2], ignore_index=True)
    
    rel_df = pd.DataFrame(rel.edges(data=True), columns=['From','To','relationship_score'])
    rel_df['relationship_score'] = rel_df['relationship_score'].map(lambda x: x['relationship_score'])

    df = pd.merge(rel_df, G_df, how = 'left', left_on=['From','To'], right_on=['From','To'])
    df = df.fillna(0)
    
    return df['movie_score'].corr(df['relationship_score'])

In [64]:
answer_four()

0.78839622217334737